In [ ]:
import numpy as np
import cv2
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pycocotools.coco import COCO
import os
import math

# Path to COCO dataset annotations and images
annotations_path = r"C:\Users\dnave\Downloads\COCO Dataset\coco2017\annotations\instances_train2017.json"
images_path =r"C:\Users\dnave\Downloads\COCO Dataset\coco2017\train2017"

# Load COCO dataset
coco = COCO(annotations_path)
image_ids = coco.getImgIds()

# Define DQN agent class
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.99  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self.build_model()

    def build_model(self):
        model = Sequential()
        model.add(Dense(24, activation='relu', input_shape=(self.state_size,)))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model


    '''def build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))

        return model'''
    

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.max(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Define IOU function to compute Intersection over Union (IoU) for reward calculation
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Define the environment interaction function
def get_training_data():
    images = []
    bbs = []
    labels = []
    for img_id in image_ids[:10]:  # Load 10 images for training (for example)
        img_info = coco.loadImgs(img_id)[0]
        img_path = f"{images_path}/{img_info['file_name']}"
        img = cv2.imread(img_path)
        if img is None:
            continue
        img_resized = cv2.resize(img, (224, 224))

        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        for ann in anns:
            bbox = ann['bbox']
            x1, y1, width, height = bbox
            x2, y2 = x1 + width, y1 + height
            bbox_converted = [x1, y1, x2, y2]

            images.append(img_resized)
            bbs.append(bbox_converted)
            labels.append(ann['category_id'])

    return np.array(images), np.array(bbs), np.array(labels)

# Initialize agent and training data
state_size = 224 * 224 * 3  # Flatten image size (224x224x3)
action_size = 10  # Assume there are 10 possible bounding boxes to choose from (as an example)
agent = DQNAgent(state_size, action_size)

images_train, bbs_train, labels_train = get_training_data()

# Training loop
max_episodes = 1000
batch_size = 32

for e in range(max_episodes):
    state = np.reshape(images_train[e % len(images_train)], [1, state_size])
    for time in range(100):  # Maximum steps per episode
        action = agent.act(state)
        
        # Here we simulate "correct action" with IoU as reward. In real implementation, use actual environment.
        chosen_bbox = bbs_train[action]  # Chosen bounding box by agent
        ground_truth_bbox = bbs_train[e % len(bbs_train)]  # For simplicity, use same image as ground truth
        reward = iou(chosen_bbox, ground_truth_bbox)

        # Next state (in this simplified case, it remains the same)
        next_state = state
        
        done = False  # Define when an episode ends
        agent.remember(state, action, reward, next_state, done)
        state = next_state

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

    if e % 10 == 0:
        print(f"Episode {e}/{max_episodes} - Epsilon: {agent.epsilon:.2f}")


In [ ]:
agent.model.save("dqn_object_detector.h5")


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("dqn_object_detector.h5")


In [ ]:
import cv2
import numpy as np
from pycocotools.coco import COCO
import tensorflow as tf

# Paths to your COCO dataset
annotations_path = r"C:\Users\dnave\Downloads\COCO Dataset\coco2017\annotations\instances_train2017.json"
images_path = r"C:\Users\dnave\Downloads\COCO Dataset\coco2017\train2017"

# Load COCO dataset
coco = COCO(annotations_path)

# Map category_id to object names
categories = coco.loadCats(coco.getCatIds())
category_map = {cat['id']: cat['name'] for cat in categories}

# Function to query and highlight the specified object
def highlight_object(image_path, object_name, model):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print("Error loading image.")
        return
    
    image_resized = cv2.resize(image, (224, 224))  # Resize for the DQN model
    state = np.reshape(image_resized, [1, 224 * 224 * 3])  # Flatten image
    
    # Find category_id for the specified object
    category_id = [k for k, v in category_map.items() if v == object_name]
    if not category_id:
        print(f"Object '{object_name}' not found in the COCO categories.")
        return

    # Use DQN model to predict bounding box
    predicted_action = model.predict(state)
    bbox = bbs_train[int(predicted_action[0])]  # Assuming bbox is stored during training

    # Highlight the bounding box on the image
    x1, y1, x2, y2 = map(int, bbox)
    highlighted_image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    highlighted_image = cv2.putText(
        highlighted_image, 
        object_name, 
        (x1, y1 - 10), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        0.9, 
        (0, 255, 0), 
        2
    )

    # Display the image
    cv2.imshow("Highlighted Object", highlighted_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Load your trained model
agent = DQNAgent(state_size=224 * 224 * 3, action_size=10)  # Reuse your DQNAgent class
agent.model = tf.keras.models.load_model("dqn_object_detector.h5")

# Test the function
test_image_path = r"C:\Users\dnave\Downloads\COCO Dataset\coco2017\train2017\example_image.jpg"  # Replace with an actual image path
highlight_object(test_image_path, "person", agent.model)


In [ ]:
def highlight_object(image_path, object_name, model):
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print("Error loading image. Check the path:", image_path)
        return
    
    print("Image loaded successfully.")
    image_resized = cv2.resize(image, (224, 224))
    state = np.reshape(image_resized, [1, 224 * 224 * 3])

    # Find category_id
    category_id = [k for k, v in category_map.items() if v == object_name]
    if not category_id:
        print(f"Object '{object_name}' not found in COCO categories.")
        return

    print(f"Category ID for {object_name}: {category_id}")

    # Predict bounding box
    predicted_action = model.predict(state)
    print("Model prediction:", predicted_action)
    
    if len(bbs_train) <= int(predicted_action[0]):
        print("Predicted action index out of range.")
        return

    bbox = bbs_train[int(predicted_action[0])]
    print("Predicted bounding box:", bbox)

    # Draw rectangle
    try:
        x1, y1, x2, y2 = map(int, bbox)
        highlighted_image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        highlighted_image = cv2.putText(
            highlighted_image, 
            object_name, 
            (x1, y1 - 10), 
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.9, 
            (0, 255, 0), 
            2
        )
        print("Bounding box drawn successfully.")
    except Exception as e:
        print("Error drawing bounding box:", e)
        return

    # Display the image
    cv2.imshow("Highlighted Object", highlighted_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
